# Study of Moller polarimetry with the Hall D tagger

Analysis of results from a large sample of 12GeV electrons simulated in the Hall D tagger using gxtwist. The Moller target coexists in the simulation with a standard GlueX amorphous target of 10um amorphous carbon (7e-5 radiation lengths) located at the target ladder position just downstrea of the goniometer. The Moller target is mounted at 30 degrees from the vertical at a position just before the entrance to the tagger vacuum box. Two thicknesses were simulated for the Moller target in two separate simulations:

1) set 5000: 10M events/run * 1000 runs = 10B electrons at 12 GeV, 100um iron target
2) set 6000: 100M events/run * 1000 runs = 100B electrons at 12 GeV, 10um iron target

Other than the total number of electrons thrown and the thickness of the Moller target, the two simulations were carried out under the same conditions.

In [1]:
import sys
import random
import uproot
import ROOT
%jsroot on
import os
os.chdir("/srv/jupyter/gxtwist studies")
import pyxrootd.client as xclient
import numpy as np
#%pip install --user "gluex.jupyroot>=1.0.10"
from gluex.jupyroot.treeview import treeview

In [2]:
import dask.distributed
import dask
dclient = dask.distributed.Client(n_workers=50, threads_per_worker=1, memory_limit="20GB", dashboard_address='0.0.0.0:8786')

In [3]:
xrdurl = "root://cn445.storrs.hpc.uconn.edu"
xrdpath = "/Gluex/resilient/simulation/moller-2-2024/"
rfile1 = ROOT.TFile.Open(xrdurl + xrdpath + "moller_5000.root")
treekey = {'tagm': 'h1', 'tagh': 'h2', 'det7': 'h5', 'det8': 'h6'}
for i in range(7):
    treekey[f"det{i}"] = f"h{i+7}"
tree = {key: rfile1.Get(treekey[key]) for key in treekey}
chain5a = {key: ROOT.TChain(tree[key].GetName(), tree[key].GetTitle()) for key in treekey}
chain5b = {key: ROOT.TChain(tree[key].GetName(), tree[key].GetTitle()) for key in treekey}
chain6a = {key: ROOT.TChain(tree[key].GetName(), tree[key].GetTitle()) for key in treekey}
chain6b = {key: ROOT.TChain(tree[key].GetName(), tree[key].GetTitle()) for key in treekey}
xfs = xclient.FileSystem(xrdurl)
keys = {key: treekey[key] for key in treekey}
chain = chain5a
for f in xfs.dirlist(xrdpath)[1]['dirlist']:
    if len(chain['tagm'].GetListOfFiles()) == 500:
        chain = chain5b
    if "moller_5" in f['name']:
        [chain[key].Add(xrdurl + xrdpath + f['name']) for key in keys]
keys = {key: treekey[key] for key in treekey}
chain = chain6a
for f in xfs.dirlist(xrdpath)[1]['dirlist']:
    if len(chain['tagm'].GetListOfFiles()) == 500:
        chain = chain6b
    if "moller_6" in f['name']:
        [chain[key].Add(xrdurl + xrdpath + f['name']) for key in keys]
tview = {}
for cname,chain in {"chain5a": chain5a, "chain5b": chain5b, "chain6a": chain6a, "chain6b": chain6b}.items():
    [print(cname, key, "created with", chain[key].GetNtrees(), "files") for key in treekey]
    tview[cname] = {key: treeview(chain[key], f"moller-sim56.root/{cname}.{key}") for key in treekey}
    [tview[cname][key].enable_dask_cluster(dclient) for key in treekey]
tree['tagm'].Print()

chain5a tagm created with 500 files
chain5a tagh created with 500 files
chain5a det7 created with 500 files
chain5a det8 created with 500 files
chain5a det0 created with 500 files
chain5a det1 created with 500 files
chain5a det2 created with 500 files
chain5a det3 created with 500 files
chain5a det4 created with 500 files
chain5a det5 created with 500 files
chain5a det6 created with 500 files
chain5b tagm created with 500 files
chain5b tagh created with 500 files
chain5b det7 created with 500 files
chain5b det8 created with 500 files
chain5b det0 created with 500 files
chain5b det1 created with 500 files
chain5b det2 created with 500 files
chain5b det3 created with 500 files
chain5b det4 created with 500 files
chain5b det5 created with 500 files
chain5b det6 created with 500 files
chain6a tagm created with 500 files
chain6a tagh created with 500 files
chain6a det7 created with 500 files
chain6a det8 created with 500 files
chain6a det0 created with 500 files
chain6a det1 created with 50

In [4]:
MeV = 1e-3 #GeV
def scale_errors(h1d, factor=1):
    """
    Rescale the errors on the contents of a 1d histogram by factor.
    """
    for i in range(h1d.GetNbinsX()):
        h1d.SetBinError(i+1, h1d.GetBinError(i+1) * factor)

## 1. Rates in the tagging counters

In [5]:
def tagm_rate_hinit():
    h = {}
    h['htagm'] = ROOT.TH1D("htagm", "hits per microscope column", 1200, 730, 850)
    h['htagm'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagm'].GetYaxis().SetTitle("counts")
    h['htagm_dEmin'] = ROOT.TH1D("htagm_dEmin", "hits per microscope column, dE>dEmin", 1200, 730, 850)
    h['htagm_dEmin'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagm_dEmin'].GetYaxis().SetTitle("counts")
    h['htagm_dEmin'].SetLineColor(ROOT.kOrange + 7)
    h['htagmdE'] = ROOT.TH1D("htagmdE", "microscope pulse height", 100, 0, 10)
    h['htagmdE'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htagmdE'].GetYaxis().SetTitle("counts")
    h['htagmdE_dEmin'] = ROOT.TH1D("htagmdE_dEmin", "microscope pulse height, dE>dEmin", 100, 0, 10)
    h['htagmdE_dEmin'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htagmdE_dEmin'].GetYaxis().SetTitle("counts")
    h['htagmdE_dEmin'].SetLineColor(ROOT.kOrange + 7)
    h['tagm_tags'] = ROOT.TTree("tagm_tags", "")
    eventid = np.array([0,0], dtype=np.uint32)
    fp_yzE = np.array([0,0,0], dtype=np.float32)
    h['tagm_tags'].Branch("eventid", eventid, "eventid[2]/i")
    h['tagm_tags'].Branch("fp_yzE", fp_yzE, "fp_yzE[3]/F")
    return h
def tagm_rate_hfill(row, histos, dEmin=2*MeV):
    histos['htagm'].Fill((row.xin[2]+row.xout[2])/2)
    histos['htagmdE'].Fill(row.desum/MeV)
    if row.desum > dEmin:
        histos['htagm_dEmin'].Fill((row.xin[2]+row.xout[2])/2)
        histos['htagmdE_dEmin'].Fill(row.desum/MeV)
        filename = row.GetCurrentFile().GetName()
        n = len(filename)
        eventid = np.array([int(filename[n-9:n-5]),row.eventno], dtype=np.uint32)
        fp_yzE = np.array([row.xin[1], row.xin[2], row.pin[3]], dtype=np.float32)
        histos['tagm_tags'].SetBranchAddress("eventid", eventid)
        histos['tagm_tags'].SetBranchAddress("fp_yzE", fp_yzE)
        histos['tagm_tags'].Fill()
    return histos

def tagh_rate_hinit():
    h = {}
    h['htagh'] = ROOT.TH1D("htagh", "hits per hodooscope counter", 1200, 300, 1500)
    h['htagh'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagh'].GetYaxis().SetTitle("counts")
    h['htagh_dEmin'] = ROOT.TH1D("htagh_dEmin", "hits per hodooscope counter, dE>dEmin", 1200, 300, 1500)
    h['htagh_dEmin'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['htagh_dEmin'].GetYaxis().SetTitle("counts")
    h['htagh_dEmin'].SetLineColor(ROOT.kOrange + 7)
    h['htaghdE'] = ROOT.TH1D("htaghdE", "hodoscope pulse height", 100, 0, 10)
    h['htaghdE'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htaghdE'].GetYaxis().SetTitle("counts")
    h['htaghdE_dEmin'] = ROOT.TH1D("htaghdE_dEmin", "hodoscope pulse height, dE>dEmin", 100, 0, 10)
    h['htaghdE_dEmin'].GetXaxis().SetTitle("#Delta E (MeV)")
    h['htaghdE_dEmin'].GetYaxis().SetTitle("counts")
    h['htaghdE_dEmin'].SetLineColor(ROOT.kOrange + 7)
    h['tagh_tags'] = ROOT.TTree("tagh_tags", "")
    eventid = np.array([0,0], dtype=np.uint32)
    fp_yzE = np.array([0,0,0], dtype=np.float32)
    h['tagh_tags'].Branch("eventid", eventid, "eventid[2]/i")
    h['tagh_tags'].Branch("fp_yzE", fp_yzE, "fp_yzE[3]/F")
    return h
def tagh_rate_hfill(row, histos, dEmin=0.4*MeV):
    histos['htagh'].Fill((row.xin[2]+row.xout[2])/2)
    histos['htaghdE'].Fill(row.desum/MeV)
    if row.desum > dEmin:
        histos['htagh_dEmin'].Fill((row.xin[2]+row.xout[2])/2)
        histos['htaghdE_dEmin'].Fill(row.desum/MeV)
        filename = row.GetCurrentFile().GetName()
        n = len(filename)
        eventid = np.array([int(filename[n-9:n-5]),row.eventno], dtype=np.uint32)
        fp_yzE = np.array([row.xin[1], row.xin[2], row.pin[3]], dtype=np.float32)
        histos['tagh_tags'].SetBranchAddress("eventid", eventid)
        histos['tagh_tags'].SetBranchAddress("fp_yzE", fp_yzE)
        histos['tagh_tags'].Fill()
    return histos

def det7_rate_hinit():
    h = {}
    h['hdet7'] = ROOT.TH1D("hdet7", "e-hits at focal plane", 1200, 300, 1500)
    h['hdet7'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['hdet7'].GetYaxis().SetTitle("counts")
    h['hdet7_Emin'] = ROOT.TH1D("hdet7_Emin", "e-hits per hodooscope counter, E>Emin", 1200, 300, 1500)
    h['hdet7_Emin'].GetXaxis().SetTitle("distance along focal plane (cm)")
    h['hdet7_Emin'].GetYaxis().SetTitle("counts")
    h['hdet7_Emin'].SetLineColor(ROOT.kOrange + 7)
    h['hdet7E'] = ROOT.TH1D("hdet7E", "e-energy at focal plane", 100, -2, 4)
    h['hdet7E'].GetXaxis().SetTitle("log10(E/MeV)")
    h['hdet7E'].GetYaxis().SetTitle("counts")
    h['hdet7E_Emin'] = ROOT.TH1D("hdet7E_Emin", "e-energy at focal plane, E>Emin", 100, -2, 4)
    h['hdet7E_Emin'].GetXaxis().SetTitle("log(E/MeV)")
    h['hdet7E_Emin'].GetYaxis().SetTitle("counts")
    h['hdet7E_Emin'].SetLineColor(ROOT.kOrange + 7)
    h['det7_tags'] = ROOT.TTree("det7_tags", "")
    eventid = np.array([0,0], dtype=np.uint32)
    fp_yzE = np.array([0,0,0], dtype=np.float32)
    h['det7_tags'].Branch("eventid", eventid, "eventid[2]/i")
    h['det7_tags'].Branch("fp_yzE", fp_yzE, "fp_yzE[3]/F")
    return h
det7_tags = {}
def det7_rate_hfill(row, histos, Emin=100*MeV):
    histos['hdet7'].Fill((row.xin[2]+row.xout[2])/2)
    histos['hdet7E'].Fill(np.log10(row.pin[3]/MeV))
    if row.kind == 3 and row.pin[3] > Emin:
        histos['hdet7_Emin'].Fill((row.xin[2]+row.xout[2])/2)
        histos['hdet7E_Emin'].Fill(np.log10(row.pin[3]/MeV))
        filename = row.GetCurrentFile().GetName()
        n = len(filename)
        eventid = np.array([int(filename[n-9:n-5]),row.eventno], dtype=np.uint32)
        fp_yzE = np.array([row.xin[1], row.xin[2], row.pin[3]], dtype=np.float32)
        histos['det7_tags'].SetBranchAddress("eventid", eventid)
        histos['det7_tags'].SetBranchAddress("fp_yzE", fp_yzE)
        histos['det7_tags'].Fill()
    return histos

for chain in ("chain5a", "chain5b", "chain6a", "chain6b"):
    tview[chain]['tagm'].declare_histograms("microscope hits", tagm_rate_hinit, tagm_rate_hfill)
    tview[chain]['tagh'].declare_histograms("hodoscope hits", tagh_rate_hinit, tagh_rate_hfill)
    tview[chain]['det7'].declare_histograms("focal plane hits", det7_rate_hinit, det7_rate_hfill)

In [6]:
dclient_used = False
for chain in ("chain5a", "chain5b", "chain6a", "chain6b"):
    if dclient_used and False:
        dclient.reset()
    dclient_used = True
    for det in ("tagm", "tagh", "det7"):
        tview[chain][det].fill_histograms(accumsize=2)
    tview[chain]['tagm'].draw([[['htagm', 'htagm_dEmin'], ['htagmdE', 'htagmdE_dEmin']]], stats="ien")
    tview[chain]['tagh'].draw([[['htagh', 'htagh_dEmin'], ['htaghdE', 'htaghdE_dEmin']]], stats="ien")
    tview[chain]['det7'].draw([[['hdet7', 'hdet7_Emin'], ['hdet7E', 'hdet7E_Emin']]], stats="ien")
    print("Number of microscope hits over threshold was", tview[chain]['tagm'].get('htagmdE_dEmin').GetEntries(),
          "from", tview[chain]['tagm'].get('tagm_tags').GetEntries(), "events")
    print("Number of hodoscope hits over threshold was", tview[chain]['tagh'].get('htaghdE_dEmin').GetEntries(),
          "from", tview[chain]['tagh'].get('tagh_tags').GetEntries(), "events")
    print("Number of focal plane electrons with E>100MeV was", tview[chain]['det7'].get('hdet7E_Emin').GetEntries(), 
          "from", tview[chain]['det7'].get('det7_tags').GetEntries(), "events")

fill_histograms read a total of 500 tree files, 11078821.0 records
fill_histograms read a total of 500 tree files, 33763533.0 records
fill_histograms read a total of 500 tree files, 87788633.0 records
Number of microscope hits over threshold was 3579870.0 from 3579870 events
Number of hodoscope hits over threshold was 21980643.0 from 21980643 events
Number of focal plane electrons with E>100MeV was 44757089.0 from 44757089 events
fill_histograms read a total of 500 tree files, 11073889.0 records
fill_histograms read a total of 500 tree files, 33756909.0 records
fill_histograms read a total of 500 tree files, 87782384.0 records
Number of microscope hits over threshold was 3578999.0 from 3578999 events
Number of hodoscope hits over threshold was 21972652.0 from 21972652 events
Number of focal plane electrons with E>100MeV was 44746311.0 from 44746311 events
fill_histograms read a total of 500 tree files, 11446374.0 records
fill_histograms read a total of 500 tree files, 34937261.0 record

## 2. Coincidences between focal plane counters

Mollers are detected as pairs of electrons that are detected in the same RF bucket in different tagging counters, whose total tagger energy sums equals the endpoint energy. To look for such pairs, I collect a sample of events where all tagger hits coming from the same rf bucket are plotted pair-wise in search of a peak in the energy correlation plot corresponding to the Moller constraint on their summed tagger energy.

To construct a collection of simulation events corresponding to a single RF bucket, I start with a selected event and then pick at random $M-1$ other events from the full set of simulated events, where $M$~Poisson$(\mu)$. The mean count $\mu$ equals the mean number of electrons per RF bucket $I_e \left( \frac{10^{-9}}{1.6\times 10^{-19}}\right) \tau_{RF}$ where $I_e$ is the electron beam current in nA and $\tau_{RF}$ is the RF period of the beam in seconds. Rather than pick $M-1$ events from a collection where most of the events never produced a tagger hit, I replace $\mu$ with $\mu_R = \mu N_{hit}/N_{sim}$ and then select $M_R-1$ events from the simulated events with tags. 

In [61]:
fdet7_tags = uproot.open("moller-sim56.root")
det7_tags = [fdet7_tags[f'chain6{s}.det7/det7_tags'] for s in ('a','b')]
det7_eventid = np.concatenate([tags.arrays(['eventid'], library='np')['eventid'] for tags in det7_tags])
runsize = 100000000
eventid = np.uint64(det7_eventid[:,0])*runsize+det7_eventid[:,1]
ueventid = np.unique(eventid)
print("out of", eventid.shape[0], "rows, there are", ueventid.shape[0], "unique events")

out of 92870970 rows, there are 92762040 unique events


In [62]:
def make_moller_hists(Ibeam_nA, tags, moller_hit_fraction = 93e6 / 1e11, ebeam_bunch_period_s = 4e-9, maxevents=1e99):
    etags_per_bunch = Ibeam_nA * (1e-9/1.6e-19) * ebeam_bunch_period_s * moller_hit_fraction   
    hname = f"h2EvsE_{Ibeam_nA}"
    htitle = f"E vs E of all tag pairs in one bunch at {Ibeam_nA}nA"
    h2EvsE = ROOT.TH2D(hname, htitle, 120, 3, 9, 120, 3, 9)
    hname2 = f"h2yvsy_{Ibeam_nA}"
    htitle2 = f"yfp vs yfp of all tag pairs in one bunch at {Ibeam_nA}nA"
    h2yvsy = ROOT.TH2D(hname2, htitle2, 120, -3, 3, 120, -3, 3)
    hname3 = f"h2zvsz_{Ibeam_nA}"
    htitle3 = f"zfp vs zfp of all tag pairs in one bunch at {Ibeam_nA}nA"
    h2zvsz = ROOT.TH2D(hname3, htitle3, 200, 400, 1400, 200, 400, 1400)
    h2EvsE.SetDirectory(0)
    h2yvsy.SetDirectory(0)
    h2zvsz.SetDirectory(0)
    ientry = 0
    nentries = min(tags['eventid'].shape[0], maxevents)
    for it in range(nentries):
        ntags = np.random.poisson(etags_per_bunch)
        while ntags < 1:
            if etags_per_bunch < 1e-3:
                ntags = 1 + np.random.poisson(etags_per_bunch/2)
            else:
                nthrow = 1 + int(1 / etags_per_bunch)
                ntags = np.random.poisson(etags_per_bunch, size=nthrow)
                nonzeros = np.flatnonzero(ntags)
                if len(nonzeros) > 0:
                    ntags = ntags[nonzeros[0]]
                else:
                    ntags = 0
        Etags = []
        ytags = []
        ztags = []
        while ntags > 0 and ientry < nentries:
            ntags -= 1
            ievent = tags['eventid'][ientry,1]
            while ientry < nentries and tags['eventid'][ientry][1] == ievent:
                ytags.append(tags['fp_yzE'][ientry,0])
                ztags.append(tags['fp_yzE'][ientry,1])
                Etags.append(tags['fp_yzE'][ientry,2])
                ientry += 1
        for i,E1 in enumerate(Etags):
            y1 = ytags[i]
            z1 = ztags[i]
            for j in range(i+1, len(Etags)):
                E2 = Etags[j]
                y2 = ytags[j]
                z2 = ztags[j]
                h2EvsE.Fill(E1, E2)
                h2EvsE.Fill(E2, E1)
                h2yvsy.Fill(y1, y2)
                h2yvsy.Fill(y2, y1)
                h2zvsz.Fill(z1, z2)
                h2zvsz.Fill(z2, z1)
        if ientry >= nentries:
            break
    return (h2EvsE, h2yvsy, h2zvsz)

In [63]:
for Ibeam_nA in (1e-9, 0.1, 1, 3, 10):
    hname = f"h2EvsE_{Ibeam_nA}"
    h2EvsE = tview['chain6a']['det7'].get(hname)
    if not h2EvsE:
        print("remaking Moller histograms at beam current", Ibeam_nA, "nA")
        results = []
        nresults = 0
        for ss in ('a', 'b'):
            for tags in uproot.iterate(f"moller-sim56.root:chain6{ss}.det7/det7_tags", step_size=1000000, library='np'):
                results.append(dclient.submit(make_moller_hists, Ibeam_nA, tags))
        output = dclient.gather(results)
        histos = [histo for histo in output[0]]
        for chunk in output[1:]:
            for i in range(len(chunk)):
                histos[i].Add(chunk[i])
        for histo in histos:
            tview['chain6a']['det7'].put(histo)
            print("histogram", histo.GetName(), "has", histo.GetEntries(), "entries")

In [67]:
h2 = {}
for Ibeam_nA in (1e-9, 0.1, 1, 3, 10):
    hname = f'h2EvsE_{Ibeam_nA}'
    tview['chain6a']['det7'].draw([hname for i in range(2)], stats='nie')
    can = tview['chain6a']['det7'].current_canvas
    can.cd(1)
    h2[hname] = tview['chain6a']['det7'].get(hname)
    h2[hname].SetContour(100)
    h2[hname].GetXaxis().SetTitle("e- energy (GeV)")
    h2[hname].GetXaxis().SetTitle("e- energy (GeV)")
    h2[hname].Draw("colz")
    can.cd(2)
    hname1 = f"{hname}1"
    htitle1 = "energy sum" + h2[hname].GetTitle()[6:]
    nbins = h2[hname].GetNbinsX()
    hsum = ROOT.TH1D(hname1, htitle1, nbins+1, 5.95, 17.95)
    hsum.GetXaxis().SetTitle("sum of tagger energies of pair (GeV)")
    hsum.GetYaxis().SetTitle("coincidences")
    hsum.SetStats(0)
    for i in range(nbins):
        for j in range(nbins):
            Esum = (h2[hname].GetXaxis().GetBinCenter(i+1) +
                    h2[hname].GetYaxis().GetBinCenter(j+1))
            counts = h2[hname].GetBinContent(i+1,j+1)
            if i+j == nbins-2:
               counts *= 2
            hsum.Fill(Esum,counts)
    for i in range(hsum.GetNbinsX()):
        hsum.SetBinError(i+1, 2*hsum.GetBinContent(i+1)**0.5)
    hsum.SetMinimum(0)
    hsum.Draw()
    h2[hname1] = hsum

In [69]:
for Ibeam_nA in (1e-9, 0.1, 1, 3, 10):
    hname = f'h2yvsy_{Ibeam_nA}'
    tview['chain6a']['det7'].draw([hname for i in range(2)], stats='nie')
    can = tview['chain6a']['det7'].current_canvas
    can.cd(1)
    h2[hname] = tview['chain6a']['det7'].get(hname)
    h2[hname].SetContour(100)
    h2[hname].GetXaxis().SetTitle("e- y_{fp} (cm)")
    h2[hname].GetXaxis().SetTitle("e- y_{fp} (cm)")
    h2[hname].Draw("colz")
    can.cd(2)
    hname1 = f"{hname}1"
    nbins = h2[hname].GetNbinsX()
    h2masked = h2[hname].Clone(hname1)
    h2masked.SetStats(0)
    h2masked.GetXaxis().SetTitle("tag 1 y_{fp} (cm)")
    h2masked.GetYaxis().SetTitle("tag 2 y_{fp} (cm)")
    for i in range(int(nbins/2), int(nbins/2 + 4)):
        for j in range(nbins):
            h2masked.SetBinContent(i, j+1, 0)
            h2masked.SetBinContent(j+1, i, 0)
    h2masked.Draw('colz')
    h2[hname1] = h2masked

In [66]:
h2 = {}
for Ibeam_nA in (1e-9, 0.1, 1, 3, 10):
    hname = f'h2zvsz_{Ibeam_nA}'
    tview['chain6a']['det7'].draw([hname for i in range(2)], stats='nie')
    can = tview['chain6a']['det7'].current_canvas
    can.cd(1)
    h2[hname] = tview['chain6a']['det7'].get(hname)
    h2[hname].SetContour(100)
    h2[hname].GetXaxis().SetTitle("tagger z position (cm)")
    h2[hname].GetXaxis().SetTitle("tagger z position (cm)")
    h2[hname].Draw("colz")
    can.cd(2)
    hname1 = f"{hname}1"
    nbins = h2[hname].GetNbinsX()
    h2masked = h2[hname].Clone(hname1)
    for i in range(nbins):
        for j in range(i-2, i+4):
            h2masked.SetBinContent(i+1, j+1, 0)
            h2masked.SetBinContent(j+1, i+1, 0)
    for i in range(20):
        for j in range(20):
            h2masked.SetBinContent(i+1, j+1, 0)
    for i in range(100, nbins):
        for j in range(270-i, nbins):
            h2masked.SetBinContent(i+1, j+1, 0)
    h2masked.Draw('colz')
    h2[hname1] = h2masked

In [11]:
import ROOT
f = ROOT.TFile("moller-sim56.root", "update")
f.cd("chain6a.det7")
print(ROOT.gDirectory)
#ROOT.gDirectory.Delete("h2EvsE_*;*")
ROOT.gDirectory.ls()

Name: chain6a.det7 Title: chain6a.det7
TDirectoryFile*		chain6a.det7	chain6a.det7
 KEY: TH1D	hdet7;1	e-hits at focal plane
 KEY: TH1D	hdet7_Emin;1	e-hits per hodooscope counter, E>Emin
 KEY: TH1D	hdet7E;1	e-energy at focal plane
 KEY: TH1D	hdet7E_Emin;1	e-energy at focal plane, E>Emin
 KEY: TTree	det7_tags;23	 [current cycle]
 KEY: TTree	det7_tags;22	 [backup cycle]
 KEY: TH1D	fill_histograms_stats;1	file processing statistics
 KEY: TH2D	h2EvsE_1e-09;2	E vs E of all tag pairs in one bunch at 1e-09nA [current cycle]
 KEY: TH2D	h2EvsE_1e-09;1	E vs E of all tag pairs in one bunch at 1e-09nA [backup cycle]
 KEY: TH2D	h2EvsE_0.1;2	E vs E of all tag pairs in one bunch at 0.1nA [current cycle]
 KEY: TH2D	h2EvsE_0.1;1	E vs E of all tag pairs in one bunch at 0.1nA [backup cycle]
 KEY: TH2D	h2EvsE_1;2	E vs E of all tag pairs in one bunch at 1nA [current cycle]
 KEY: TH2D	h2EvsE_1;1	E vs E of all tag pairs in one bunch at 1nA [backup cycle]
 KEY: TH2D	h2EvsE_3;2	E vs E of all tag pairs in one bun